# Prepare Notebook Environment for Biomarker Agents
In this section we prepare this notebook environment with the necessary dependencies to create all Biomarker agents. You only need to run through this notebook, utils.ipynb simply contains imports that will be used throughout the multi-agent development process.

#### Run the pip3 commands below to install all needed packages

In [ ]:
!pip3 install -r requirements.txt
!pip3 install --upgrade boto3
!pip3 show boto3

#### Ensure the latest version of boto3 is shown below

In [ ]:
!pip freeze | grep boto3

#### Import all needed Python libraries

In [ ]:
IMPORTS_PATH = "/home/ec2-user/SageMaker/multi_agent_development/0-Notebook-environment/utils.ipynb"
%store IMPORTS_PATH

In [ ]:
%run $IMPORTS_PATH

#### Extract account information needed for agent creation and define needed agent models

In [ ]:
# boto3 session
sts_client = boto3.client('sts')
session = boto3.session.Session()

# Account info
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

# FM used for all Biomarker sub-agents, this version needed for agent collaboration
agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]

# Supervisor agent needs cross-inference model to avoid throttling issues
supervisor_agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]

#### Store all needed variables in environment for future use in development

In [ ]:
# Store account info
%store account_id
%store region

# Store model lists
%store agent_foundation_model
%store supervisor_agent_foundation_model

#### Navigate to the SageMaker Notebook execution role displayed below in IAM and attach the following policies: 

##### ["BedrockFullAccess", "IAMFullAccess", "AmazonSSMFullAccess", "AWSLambda_FullAccess"]

Give a few minutes for these permissions to update

In [ ]:
# Output SageMaker Notebook execution role for user to add policies to

# Get the SageMaker session
sagemaker_session = sagemaker.Session()

# Get the execution role
role = sagemaker_session.get_caller_identity_arn()

print(f"SageMaker Execution Role: {role}")

#### Ensure that you have access to Bedrock models needed by the agents
If you have to enable model access, give a couple minutes before proceeding with agent creation

In [ ]:
# Define bedrock client
bedrock_client = boto3.client('bedrock', region)

# List the foundation models in account
response = bedrock_client.list_foundation_models()

# Extract model names
model_summaries = response['modelSummaries']
model_names = [model['modelName'] for model in model_summaries]

# Ensures that this list has the models 'Claude Sonnet 3.5' and 'Claude 3.5 Sonnet v2' for agent use

if 'Claude Sonnet 3.5' and 'Claude 3.5 Sonnet v2' in model_names:
    print("You have access to all models needed!")
else:
    print("Enable access to both 'Claude Sonnet 3.5' and 'Claude 3.5 Sonnet v2' models using the Bedrock console")